### Transformer Theoretical Model

This notebook stores a bunch of analysis about a Transformer, e.g. estimates the number of FLOPs, parameters, peak memory footprint, checkpoint size, etc.

In [ ]:
from collections import OrderedDict

In [ ]:
# config_args = {
#     'gpt2':         dict(n_layer=12, n_head=12, n_embd=768),  # 124M params
#     'gpt2-medium':  dict(n_layer=24, n_head=16, n_embd=1024), # 350M params
#     'gpt2-large':   dict(n_layer=36, n_head=20, n_embd=1280), # 774M params
#     'gpt2-xl':      dict(n_layer=48, n_head=25, n_embd=1600), # 1558M params
# }[model_type]

state_dim = 1280
action_dim = 1858 + 62 # padding for multiple of 128 if I decide to do this

window_size = 42
window_padding = 2
seq_size = window_size*3 # + window_padding if I decide to do this

max_ep_len = 1024
n_layer = 8
n_head = 8
n_embd = 64 * n_head

ffw_size = 4*n_embd # feed forward size - suggested 4*n_embd

batch_size = 5120 # total batch size is batch_size * grad_accum
# estimated rn, could find out by running the model
measured_time = 2.1 # in seconds per iteration - AK recorded 0.755s: A100s are 2.78x faster than V100s, so 0.755*2.78 = 2.1s

epochs = 100
chess_games = 16e6 # 16M games per epoch


bias = False
assert not bias, "this notebook assumes bias=False just for simplicity"

In [ ]:
def params():
    """ estimates the number of parameters in the model"""
    out = OrderedDict()

    # token and position embeddings
    out['____embedding/timesteps'] = n_embd * max_ep_len
    out['____embedding/state'] = n_embd * state_dim
    out['____embedding/action'] = n_embd * action_dim
    out['____embedding/return'] = n_embd
    out['embedding'] = out['____embedding/timesteps'] + out['____embedding/state'] + out['____embedding/action'] + out['____embedding/return']

    # attention blocks
    out['____attention/ln'] = n_embd # note, bias=False in our LN
    out['____attention/kqv'] = n_embd * 3*n_embd
    out['____attention/proj'] = n_embd**2
    out['__attention'] = out['____attention/ln'] + out['____attention/kqv'] + out['____attention/proj']

    # MLP blocks
    out['____mlp/ln'] = n_embd
    out['____mlp/ffw'] = n_embd * ffw_size
    out['____mlp/proj'] = ffw_size * n_embd
    out['__mlp'] = out['____mlp/ln'] + out['____mlp/ffw'] + out['____mlp/proj']

    # the transformer and the rest of it
    out['__block'] = out['__attention'] + out['__mlp']
    out['transformer'] = n_layer * out['__block']
    out['ln_f'] = n_embd # final layernorm

    out['____prediction/state'] = n_embd * state_dim
    out['____prediction/action'] = n_embd * action_dim
    out['____prediction/return'] = n_embd
    out['prediction'] = out['____prediction/state'] + out['____prediction/action'] + out['____prediction/return']

    # total
    out['total'] = out['embedding'] + out['transformer'] + out['ln_f'] + out['prediction']

    return out

# compare our param count to that reported by PyTorch
p = params()
params_total = p['total']
print(f"we see: {params_total}, expected: {124337664}, match: {params_total == 124337664}")
# create a header
print(f"{'name':20s} {'params':10s} {'ratio (%)':10s}")
for k,v in p.items():
    print(f"{k:20s} {v:10d} {v/params_total*100:10.4f}")


In [ ]:
# we can now calculate the size of each checkpoint
# params are stored in fp32, and the AdamW optimizer has 2 additional buffers per param for statistics
params_bytes = params_total*4
params_and_buffers_bytes = params_bytes + 2*params_bytes
print(f"est checkpoint size: {params_and_buffers_bytes/1e9:.2f} GB")
# measured_bytes = 1542470366 # from wc -c ckpt.pt
# print(f"measured with wc -c ckpt.pt: {measured_bytes}")
# print(f"fluff ratio: {measured_bytes/params_and_buffers_bytes*100:.2f}%")

We can also estimate the ratio of our GPU memory that will be taken up just by the weights and the buffers inside the AdamW optimizer

In [ ]:
gpu_memory = 32e9 # 32 GB V100 GPU, roughly
print(f"memory ratio taken up just for parameters: {params_and_buffers_bytes / gpu_memory * 100:.2f}%")

i.e. not that much of the memory for this tiny model, most of the memory is activations (forward and backward). This of course changes dramatically for larger and larger models.

Let's estimate FLOPs for a single forward pass.

In [ ]:
def flops():
    # we only count Weight FLOPs, all other layers (LayerNorm, Softmax, etc) are effectively irrelevant
    # we count actual FLOPs, not MACs. Hence 2* all over the place
    # basically for any matrix multiply A (BxC) @ B (CxD) -> (BxD) flops are 2*B*C*D

    out = OrderedDict()
    head_size = n_embd // n_head

    # attention blocks
    # 1) the projection to key, query, values
    out['____attention/kqv'] = 2 * seq_size * (n_embd * 3*n_embd)
    # 2) calculating the attention scores
    out['____attention/scores'] = 2 * seq_size * seq_size * n_embd
    # 3) the reduction of the values (B, nh, T, T) x (B, nh, T, hs) -> (B, nh, T, hs)
    out['____attention/reduce'] = 2 * n_head * (seq_size * seq_size * head_size)
    # 4) the final linear projection
    out['____attention/proj'] = 2 * seq_size * (n_embd * n_embd)
    out['__attention'] = sum(out['____attention/'+k] for k in ['kqv', 'scores', 'reduce', 'proj'])

    # MLP blocks
    ffw_size = 4*n_embd # feed forward size
    out['____mlp/ffw1'] = 2 * seq_size * (n_embd * ffw_size)
    out['____mlp/ffw2'] = 2 * seq_size * (ffw_size * n_embd)
    out['__mlp'] = out['____mlp/ffw1'] + out['____mlp/ffw2']

    # the transformer and the rest of it
    out['block'] = out['__attention'] + out['__mlp']
    out['transformer'] = n_layer * out['block']

    out['__prediction/state'] = 2 * window_size * (n_embd * state_dim)
    out['__prediction/action'] = 2 * window_size * (n_embd * action_dim)
    out['__prediction/return'] = 2 * window_size * n_embd
    out['prediction'] = out['__prediction/state'] + out['__prediction/action'] + out['__prediction/return']

    # forward,backward,total
    out['forward_total'] = out['transformer'] + out['prediction']
    out['backward_total'] = 2 * out['forward_total'] # use common estimate of bwd = 2*fwd
    out['total'] = out['forward_total'] + out['backward_total']

    return out

# compare our param count to that reported by PyTorch
f = flops()
flops_total = f['forward_total']
print(f"{'name':20s} {'flops':14s} {'ratio (%)':10s}")
for k,v in f.items():
    print(f"{k:20s} {v:14d} {v/flops_total*100:10.4f}")


In [ ]:
# now here is an estimate copy pasted from the PaLM paper
# this formula is often used to calculate MFU (model flops utilization)
def palm_flops():
    """estimate of the model flops following PaLM paper formula"""
    # non-embedding model parameters. note that we do not subtract the
    # embedding/token params because those are tied and get used in the last layer.
    N = params()['total'] - params()['____embedding/timesteps']
    L, H, Q, T = n_layer, n_head, n_embd//n_head, seq_size
    mf_per_token = 6*N + 12*L*H*Q*T
    mf = mf_per_token * seq_size
    return mf

print(f"palm_flops: {palm_flops():d}, flops: {flops()['total']:d}, ratio: {palm_flops()/flops()['total']:.4f}")

Sam Note: Ratio is higher because, well, it's a slightly diff architecture. There are 3 predictions instead of 1, and the timestep embedding is much larger than the block_size (seq_size) param. In a regular transformer, those are the same size. Output dimension is a lot smaller, etc. I don't think this is super informative for me.

"Ok they are quite similar, giving some confidence that my math in flops() function was ~ok. Now, A100 is cited at 312TFLOPS bfloat16 on tensor cores. So what is our model flops utilization (MFU)? I trained the model above with a batch_size of 20 and grad_accum of 5, which runs in about 755ms on a single A100 GPU. We get:"

Sam Note: Well V100s is only cited at 112 TFLOPS on tensor performance

In [ ]:
# here is what we currently roughly measure
measured_throughput = batch_size / measured_time
flops_achieved = f['total'] * measured_throughput

# A100 is cited to be 312 TFLOPS of bloat16 running on tensor cores
v100_flops_promised = 112e12

# the fraction of the A100 that we are using:
print(f"fraction of V100 used: {flops_achieved / v100_flops_promised * 100:.2f}%")
assumed_mfu = 0.8*(flops_achieved / v100_flops_promised) # assume this model flops utilization (take fraction of V100 from above and add some DDP overhead by removing ~20% of it)

For reference, we'd prefer to be somewhere around 50%+, and not just for a single GPU but for an entire DDP run. So we still have some work to do, but at least we're within a factor of ~2X of what is achievable with this GPU.

In [ ]:
# Finally let's check out the 6ND approximation as total cost of training in FLOPs
model_size = params()['total'] # this is number of parameters, N
tokens_num = epochs * chess_games * seq_size # this is dataset size in tokens, D
v100_flops = 112e12 # 312 TFLOPS
flops_throughput = v100_flops * 4 * assumed_mfu # assume an 4XV100 node at mfu utilization
flops_needed = 6 * model_size * tokens_num # 6ND or 8ND with checkpointing
time_needed_s = flops_needed / flops_throughput # in seconds
print(f"time needed to train the model: {time_needed_s/3600/24:.2f} days")

This is not a bad estimate at all. I trained this model and it converged in roughly 4 days. Btw as a good reference for where 6ND comes from and some intuition around it I recommend [Dzmitry's post](https://medium.com/@dzmitrybahdanau/the-flops-calculus-of-language-model-training-3b19c1f025e4).

Sam Note: Dzmitry's eq: compute $\approx$ 6 $\times$ num_params $\times$ num_tokens trained on
- "My favorite corollary of this equation is that assuming constant throughput, the training time grows linearly with the model size. So if you want to increase the model size by 2, you have to either use 2 times as many GPUs, or wait 2 times longer. Easy mathematics that you can do in your head!" - Dzmitry

Now, FLOPs are just one constraint, the other that we have to keep a close track of is the memory bandwidth. TODO estimate LOAD/STORE costs of our model later.